# TODO

https://towardsdatascience.com/winning-blackjack-using-machine-learning-681d924f197c

1. Andmestiku puhastus, ainult need asjad mida vaja on.
2. Sorteerida ära terve andmestiku peale pärast esimese kahe kaardi andmist kas käsi on soft-hand, hard-hand või paar (nagu need tabelid üleval antud lehel on). Hard-handid saame nii kätte, et kus kaks kaarti pole kas paar või üks kahest kaardist ei ole 
3. Nüüd kui meil on ära sorteeritud käed, siis iga käega peaksime välja arvutama (I assume) võiduprotsendi järgi kas me peame hittima, stay või panust duubeldama. 
4. Ja nii arvutame välja kõikide juhtudega ehk kui sul on kolm kaarti käes, neli kaarti jne.

In [125]:
import pandas as pd
import warnings

data = pd.read_csv("blkjckhands.csv", index_col=False)
data = data.drop(columns=['Unnamed: 0', 'PlayerNo', 'dealcard2', 'dealcard3', 'dealcard4', 'dealcard5', 'plybustbeat', 'dlbustbeat','plwinamt', 'dlwinamt'])


In [126]:
cond = (data.card3 == 0) & (data.card4 != 0)
data.loc[cond, ['card3', 'card4']] = data.loc[cond,['card4', 'card3']].values



#154980 - kus on kolmas kaart on õiges kohas
#190401 - kus on kolmas kaart vales kohas
#35603 - kus on kolmas ja neljas kaart vales kohas

In [127]:
cond2 = (data.card3 != 0) & (data.card4 == 0) & (data.card5 != 0)
data.loc[cond2, ['card4', 'card5']] = data.loc[cond2, ['card5', 'card4']].values

data[(data["card3"] != 0) & (data["card4"] == 0) & (data["card5"] != 0)]

,card1,card2,card3,card4,card5,sumofcards,dealcard1,sumofdeal,blkjck,winloss,ply2cardsum


In [128]:
#Sorteerisin käed vastavalt esimese kahe kaardi järgi, sealt vastavalt saame teha maatrixi siis otsustest based on win rate
#Lisaks peab jälgima seda, et meil on andmeid kus on kolmas kaart 0 aga neljas ja viies on hit. Need peame lugema pmst hittideks,
#lihtsalt 

#Ehk siis me arvutame iga käe summa järgi ning kas sellega võideti või mitte. Lisaks vaatame dealeri kaarti. Umbes kuidagi selline on
#arvutus:
# 1. Näiteks hard_hands puhul võtame summa nagu näiteks 10. 
# 2. Loeme kõik 10ned kokku. Eraldame kõik diileri kaardi juhud ning võtame näiteks kus on sinu kaartide summa on 10 ning diileri summa on 2
# 3. Loeme kokku win% kõikidel kätel kus hitidakse ning kus ei hitida. (not sure how to make this work yet).
# 4. Paneme sinu tehtud maatriksisse lõpuks 0 asemele kas me hitime selle summaga ning selle dealeri handiga või standime.

#Sarnast algoritmi saab kasutada ka paaride ning soft_handide korral ehk me võime kasutada paaride korral näiteks alguses nn paaride maatriksit,
#ning näiteks teeme statistika selle põhjal kas splittida või mitte. Sealt me edasi mõtlema ei pea, kuna edasi läheb ta nkni otsustab kas tema käsi
#on hard_hand või mis iganes. Sama ka soft_handide põhjal.

pairs = data[data["card1"] == data["card2"]].copy()
soft_hands = data[(data["card1"] == 1) | (data["card1"] == 11) | (data["card2"] == 1) | (data["card2"] == 11)].copy()
hard_hands = data[(data["card1"] != data["card2"]) & ((data["card1"] != 1) & (data["card1"] != 11) & (data["card2"] != 1) & (data["card2"] != 11))].copy()

Otsustuse dataframe on vaja panna paika siis nende kriteerimite alusel:
1. Row tähistab handi väärtust ja veerg tähistab dealeri esimest kaarti. Me peame row väärtuseks võtma ka need handid kus ntks alles viies kaart oli see mis summaks andis 20 ehk me itereerime iga rida kus ka kaartide vaheväärtus oli otsitav summa. Ühesõnaga ütleme, et me otsime hande kus kaartide summaks oli 18, st et me vaatame hande kus esimese kahe kaardi, esimese kolme kaari, esimese nelja kaardi... jne summa oli 18 ja kas siis oli HIT v oli STAND ja kas käsi lõpuks võideti või ei.

2. Me loeme HITID ja STANDID kokku ja vaatame kumma puhul suurem võidu % oli ja kanname vastavalt siis kummagi tabelisse.

In [129]:
veerud = {'2':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '3':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '4':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '5':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '6':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '7':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '8':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '9':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         'T':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         'A':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0']}
Otsus = pd.DataFrame(veerud, index=['20','19','18','17','16','15','14','13','12','11','10','9','8','7','6','5'])
Otsus

,2,3,4,5,6,7,8,9,T,A
20,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0


In [130]:
#Teeme ühe korra arvutuse läbi
#Võtame kaartide summaks 5 ja dealeri uphandiks 2
#temp = hard_hands[(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] + hard_hands["card5"] == 5)].copy()
def hitvstand(summa,dcard):
    summa = int(summa)
    if(dcard == "A"):
        temp = hard_hands[
            ((hard_hands["ply2cardsum"] == summa)| (hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] == summa)
            |(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] == summa))
             & ((hard_hands["dealcard1"] == 1) | (hard_hands["dealcard1"] == 11))
            ].copy()
    elif(dcard == "T"):
        temp = hard_hands[
            ((hard_hands["ply2cardsum"] == summa)| (hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] == summa)
            |(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] == summa))
             & (hard_hands["dealcard1"] == 10)
            ].copy()
    else:
        temp = hard_hands[
            ((hard_hands["ply2cardsum"] == summa)| (hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] == summa)
            |(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] == summa))
             & (hard_hands["dealcard1"] == int(dcard))
            ].copy()
    
    
    standwin = temp[(temp["winloss"]=="Win") & ((temp["card1"] + temp["card2"]==summa) & (temp["card3"]==0))
                  |
                ((temp["card1"]+temp["card2"]+temp["card3"]==summa) & (temp["card3"]!=0) & (temp["card4"]==0))
                 |
                 ((temp["card1"]+temp["card2"]+temp["card3"]+temp["card4"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]==0))].copy()
    
    hitwin = temp[(temp["winloss"]=="Win") & ((temp["card1"] + temp["card2"]==summa) & (temp["card3"]!=0))
                  |
                ((temp["card1"]+temp["card2"]+temp["card3"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0))
                 |
                 ((temp["card1"]+temp["card2"]+temp["card3"]+temp["card4"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]!=0))].copy()
    

    if((len(hitwin.index)) > (len(standwin.index))):
        return 'H'
    else:
        return 'S'

Columns = list(Otsus.columns)
Rows = list(Otsus.index)
for i in range(len(Rows)):
    for j in range(len(Columns)):
        taht = hitvstand(Rows[i], Columns[j])
        Otsus.iloc[i][j] = taht
        
Otsus


,2,3,4,5,6,7,8,9,T,A
20,S,S,S,S,S,S,S,S,S,S
19,S,S,S,S,S,S,S,S,S,S
18,S,S,S,S,S,S,S,S,S,S
17,S,S,S,S,S,S,S,S,S,S
16,S,S,S,S,S,H,H,H,H,H
15,S,S,S,S,S,H,H,H,H,H
14,S,S,S,S,S,H,H,H,H,H
13,S,S,S,S,S,H,H,H,H,H
12,H,H,S,S,S,H,H,H,H,H
11,H,H,H,H,H,H,H,H,H,H
